**Pouya Sadeghi**

fall-1401(2022)

# AI course, project #2.1
**Genetic Algorithms**

# 01.Description

## Problem definition 

Given allowed operators and operand and equation-length

(which contains of $(len-1)/2$ operators and $(len+1)/2$ operands)

and target result, asking for equation than satisfy problem condition.

## Chromosomes

We define chromosomes as a collection of genes, consists of operand and operators;
each chromosome is a sequence of genes that make an equation.

Example:
- len(eq) = 11
- operands = [1, 2, 3, 4, 5, 6]
- operators = [+, /, -]

sample chromosome $->$ (1, +, 2, /, 5, -, 9, +, 3, +, 6,)
$=>$ $1+2/5-9+3+6=1$


## Hyper-parameters


### population size

We use this parameter either for first-initial-population and for generate some new random population in each iteration

    (it determine number of population for the beginning of each generation)

### carry percentage

Determine the ratio of population sorted by fitness that go straight to the next generation

### crossover probability

Determine the ration of mating-pool that would crossovers

### generate new population rate

We will add ($PopulationSize * GenerateNewPopulationRate$) new random population to our current generation population

### fitness_termination_threshold

If fitness score stuck in a value(couldn't improve) for certain iteration count, we would terminate the process,

**Note**: because of straight-forward-population, best fitness of a generation couldn't decrease in next generation.

## fitness

We define fitness as $fitness(chromosome) = -|TargetResult - ChromosomeEquationResult|$ 

With that, all fitnesses are less than or equal to zero and zero fitness is our goal.

later, we discuss how we can map this values into positive ones.

## random population

Randomly generating sequence of $operand1[i], operator[j], operand[k], ...$ 
by choosing random operand and operator

## Next Generation

In each iteration, we:
- extend population using random chromosomes and shuffle hole population
- calculate *fitness* and send certain population straight to next generation
- create mating pool:
    - respect higher fitness by increasing their population
    - peak random sample of size $DesiredPopulation - StraightToNextGeneration$
- crossover population with $P(CrossoverProbability)$
    - they would crossover or remain the same
- mutate some gene of each chromosome randomly

### crossover

for each chromosome:
- choose random partner
- select some gene by their position randomly
- swap parents genes
- add chromosome with higher *fitness* to crossover pool

### mutation

for each chromosome:
- choose random gene(s) for mutation
- for each selected gene:
    - if $type(gene) == operand$
        choose random operand to replace with
    - if $type(gene) == operator$
        choose random operator to replace with

In [1]:
import genetic

# 02.Equation

define our desired equation

In [2]:
equation = genetic.Equation(
    operands=[1, 2, 3, 4, 5, 6, 7, 8],
    operators=['+', '-', '*'],
    length=21,
    result=18019
    )

# 03.EquationBuilder

## generate agent for our problem

In [3]:
agent = genetic.EquationBuilder(
    equation,
)

## configuration

we can configure agent hyper-parameter

this parameters are the same for agents

## check configs

In [4]:
agent.get_config()

{'crossover_probability': 0.8,
 'carry_percentage': 0.25,
 'population_size': 800,
 'generate_new_population_rate': 0.1,
 'fitness_termination_threshold': 60}

# 04.Find Equation

In [5]:
built_equation, iteration = agent.find_equation()

In [6]:
if built_equation is None:
    print("Equation: Not Found")
else:
    print(f"Equation: {' '.join(built_equation)}")

Equation: 5 + 2 * 6 + 3 * 5 * 4 * 6 * 5 * 5 * 2 + 2


## check built equation

In [7]:
if equation.is_satisfied(built_equation):
    print(f"Agent solved problem correctly in {iteration} iterations")
else:
    print("Agent failed")

Agent solved problem correctly in 44 iterations


# 05.Discussion

## 01. pros and cons of high/low population?

- high population:
    - pros:
        1. we could find a solution in less iteration
        2. wide diversity
    - cons:
        1. each iteration would be longer
        2. total run time might increase because of each iteration time
- low population:
    - pros:
        1. population generation would be faster
        2. each iteration would be shorter
    - cons:
        1. we need more iteration to find a solution
        2. limited diversity
        3. total rum time would increase due to more iteration

## 02.what happens if population increase in each iteration?

*Accuracy*: due to increased diversity, would increase

*Speed*: due to computation increment, would decrease

## 03.compare crossover and mutation. can only one of them be used?

mutation :
- would change chromosomes randomly
- prevents bias
- using it alone is like try all possible scenarios (and decrease speed)

crossover:
- could cause orientation
- it's not guaranteed that it can find a solution on its own

In GAs, crossover would increase speed and mutation would increase accuracy



## 04.how could we increase speed in this problem?

- add random population in each iteration (or some of them) and prevent from orientation (bias)
- increase crossover probability (up to a threshold)
- increase `straight_to_next_generation` (up to a threshold, because it can cause orientation and cause difficulty finding a solution)
- use a suitable *fitness-function* (chromosomes score could help us in other parts, like what happened in `_make_mating_pool`)
- pay attention to mating-pool creation (how to create this pool and its parameter)

## 05.what if after some iteration the chromosomes no longer change? how could you solve that?

In this situation we aren't able to find a solution if we haven't yet.

To solve this:
- we generate some random chromosome in each iteration
- via mutation, we try to generate better random chromosome(to avoid it)
- as another approach, we could define a max-ratio for mutation, by getting close to the program termination (due to the lack of fitness), we increase this ratio


## 06.how to terminate the program if there is no answer th the problem?

Three class of solutions can be imagined:
- Decisive approach (not flexible):
    - set an alert to terminate program after a while
- semi-flexible (flexible threshold):
    - set a deadline and after passing that, let current iteration to finish and then terminate ✔
    - set a specific amount of iteration ✔
- flexible:
    - terminate if fitness doesn't improved ✅
    - terminate if an unobserved population can't be created

We can control our algorithm with those options specified above.

By default, the flexible approach would apply (you could change repetition limit in config); other semi-flexible approaches are available and ready to use.